In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

import geopandas as gpd
from eodag import EODataAccessGateway
import rioxarray as riox
import numpy as np
import folium
from folium import LayerControl
from eo_tools_dev.util import show_cog
from pathlib import Path

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
log = logging.getLogger(__name__)

## Set up parameters and output dir

In [ ]:
# change to your custom locations
data_dir = "/data/S1"
outputs_prefix="/data/res/urban-berlin"
file_aoi = "../data/Berlin.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
pth_search_results = "../data/search_Potsdam_2016_2024.geojson"
s = dag.deserialize(pth_search_results)
prm_ids = [s[0].properties['id'], s[2].properties['id']]
sec_ids = [s[1].properties['id'], s[3].properties['id']]


## Pre-process InSAR pairs

In [ ]:
# from eo_tools.S1.process import process_insar
# for prm, sec in zip(prm_ids, sec_ids):
#     primary_dir = f"{data_dir}/{prm}.zip"
#     secondary_dir = f"{data_dir}/{sec}.zip"
#     out_dir = process_insar(
#         dir_prm=primary_dir,
#         dir_sec=secondary_dir,
#         outputs_prefix=outputs_prefix,
#         aoi_name=None,
#         shp=shp,
#         pol="full",
#         write_coherence=True,
#         write_interferogram=False,        
#         write_primary_amplitude=True,
#         write_secondary_amplitude=True,
#         apply_fast_esd=False,
#         dem_upsampling=1.8,
#         dem_force_download=False,
#         dem_buffer_arc_sec=40,
#         boxcar_coherence=[5, 5],
#         filter_ifg=True,
#         multilook=[1, 4],
#         warp_kernel="bicubic",
#         cal_type="sigma",
#         clip_to_shape=True,
#     )

In [ ]:
from eo_tools.S2 import process_s2_tiles, make_s2_rgb 
s2_products = [
    "/data/S2/S2A_MSIL1C_20160909T102022_N0204_R065_T33UUU_20160909T102108.zip",
    # "/data/S2/S2A_MSIL1C_20240904T100551_N0511_R022_T33UUU_20240904T153022.zip",
    "/data/S2/S2A_MSIL1C_20160909T102022_N0204_R065_T33UVU_20160909T102108.zip",
    # "/data/S2/S2A_MSIL1C_20240904T100551_N0511_R022_T33UVU_20240904T153022.zip",
]
process_s2_tiles(s2_products, bands=["B4", "B3", "B2"] , shp=shp, aoi_name="Berlin", outputs_prefix="/data/res", force_create=True)

In [ ]:
make_s2_rgb("/data/res/S2A_L1C_2016-09-09-102022_Berlin")
make_s2_rgb("/data/res/S2A_L1C_2024-09-04-100551_Berlin")

In [ ]:
from eo_tools.S1.process import (
    geocode_and_merge_iw,
    coherence,
    apply_to_patterns_for_pair,
)
import os
out_dir_2016 = list(Path.glob(Path(outputs_prefix), "S1*2016*"))[0] / "sar"
out_dir_2024 = list(Path.glob(Path(outputs_prefix), "S1*2024*"))[0] / "sar"
out_dirs = [out_dir_2016, out_dir_2024]
for prm, sec, out_dir in zip(prm_ids, sec_ids, out_dirs):
    primary_dir = f"{data_dir}/{prm}.zip"
    secondary_dir = f"{data_dir}/{sec}.zip"
    # compute interferometric coherence
    log.info(f"processing {out_dir.parent.name}")

    # apply_to_patterns_for_pair(
    #     coherence,
    #     out_dir=out_dir,
    #     file_prm_prefix="slc_prm",
    #     file_sec_prefix="slc_sec",
    #     file_out_prefix="coh",
    #     # box_size=[3, 11],
    #     box_size=[5, 5],
    #     multilook=[1, 4],
    # )

    # custom: cross polarizations
    for iw in [1, 2, 3]:
        prm_vv = out_dir / f"slc_prm_vv_iw{iw}.tif"
        prm_vh = out_dir / f"slc_prm_vh_iw{iw}.tif"
        sec_vv = out_dir / f"slc_sec_vv_iw{iw}.tif"
        sec_vh = out_dir / f"slc_sec_vh_iw{iw}.tif"
        if os.path.exists(prm_vv) and os.path.exists(prm_vh): 
            rho_prm_vv = coherence(prm_vv, prm_vh, out_dir / f"rhop_vv_iw{iw}.tif",box_size=[5, 5] , multilook=[1, 4])
            # rho_sec_vv = coherence(sec_vv, sec_vh, out_dir / f"rhos_vv_iw{iw}.tif",box_size=[5, 5] , multilook=[1, 4])
            # rhox_vv = coherence(prm_vv, sec_vh, out_dir / f"rhox_vv_iw{iw}.tif",box_size=[5, 5] , multilook=[1, 4])
            # rhox_vh = coherence(prm_vh, sec_vv, out_dir / f"rhox_vh_iw{iw}.tif",box_size=[5, 5] , multilook=[1, 4])


    geo_dir = Path(out_dir).parent
    # geocode_and_merge_iw(input_dir=geo_dir, var_names=["coh"], shp=shp, pol="full")
    # geocode_and_merge_iw(input_dir=geo_dir, var_names=["rhop", "rhos"], shp=shp, pol="full")
    geocode_and_merge_iw(input_dir=geo_dir, var_names=["rhox"], shp=shp, pol="full")

## Compute coherence changes

## Visualize
Click on top right icon to hide/show layers

In [ ]:
from pathlib import Path
dir_products = Path(outputs_prefix).glob("S1*")

m = folium.Map()

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

import os
for d in dir_products:
    _ = show_cog(f"{d}/coh_vh.tif", m, rescale="0.2,0.8", resampling="nearest")
    # _ = show_cog(f"{d}/coh_vh.tif", m, rescale="0.2,0.8", resampling="nearest", expression="(2/(b1+1)-1)")
    # _ = show_cog(f"{d}/coh_vv.tif", m, rescale="0,1")
    # _ = show_cog(f"{d}/amp_prm_vv.tif", m, rescale="0,0.7", resampling="bilinear")
    # if os.path.exists(f"{d}/diff_coh_vv.tif"):
        # _ = show_cog(f"{d}/diff_coh_vv.tif", m, rescale="0.2,0.5")
    # if os.path.exists(f"{d}/diff_coh_vh.tif"):
        # _ = show_cog(f"{d}/diff_coh_vh.tif", m, rescale="-2,2", colormap_name="rdbu")

    # if os.path.exists(f"{d}/diff_amp_vv.tif"):
    #     _ = show_cog(f"{d}/diff_amp_vv.tif", m, rescale="-1,1", colormap_name="rdbu")
    # _ = show_cog(f"{d}/amp_prm_vh.tif", m, rescale="0,1")
LayerControl().add_to(m)
m